<a href="https://colab.research.google.com/github/Jdmolano2020/TesisNer/blob/main/CargueDatasetSROIE_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.3 MB/s eta 0:00:00


##Cargar dataset Sroie

In [ ]:
# limpiamos rastros del dataset y descomprimimos
!rm -rf /tmp/SROIE2019v02
!unzip /content/drive/MyDrive/archive_SROIEv2.zip -d /tmp/SROIE2019v02/

Archive:  /content/drive/MyDrive/archive_SROIEv2.zip
   creating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/config.json  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/pytorch_model.bin  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/special_tokens_map.json  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/tokenizer_config.json  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/training_args.bin  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/vocab.txt  
   creating: /tmp/SROIE2019v02/SROIE2019/test/
   creating: /tmp/SROIE2019v02/SROIE2019/test/box/
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X00016469670.txt  
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X00016469671.txt  
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X51005200931.txt  
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X51005230605.txt  
  inflating: /t

##Importar Librerias

In [ ]:
import pandas as pd
import os
import json
import random
import re
import joblib
from tqdm import tqdm

import spacy.cli
from spacy import displacy
from spacy.util import minibatch, compounding
from spacy.training import Example
from spacy.scorer import Scorer
import pathlib

In [ ]:
from spacy.training import offsets_to_biluo_tags
from spacy.tokenizer import Tokenizer

## Crear Modelo NER

In [ ]:
nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe("ner")

In [ ]:
tokenizer = nlp.tokenizer

## Crear Funciones

In [ ]:
def parse(line):
    fields = line.strip().split(",")
    if len(fields) == 9:
        return fields
    else:
        return fields[:8] + [",".join(fields[8:])]

def build_text(data):
    text = " ".join(data.text)
    text = text.replace("  "," ")
    return text

def entidades_total (ruta,archivo):
  print(archivo)
  with open(ruta + "box/" + archivo, encoding='utf-8', errors='ignore') as f:
    ocr = f.readlines()
  data = pd.DataFrame(
    list(map(parse, ocr)),
    columns=[*(f"coor{i}" for i in range(8)), "text"]
    )
  data=data.dropna()
  #print("data",data)
  texto = build_text(data)
  #print("texto",texto)

  with open(ruta + "entities/" + archivo, encoding='utf-8') as f:
    entities = json.load(f)
  #print("entities=",entities)
  #print("type_entities=",type(entities))
  entidades = []
  for name, ent_text in entities.items():
    if len(ent_text)==0:
      continue
    else:
      ent_text=ent_text.translate(str.maketrans({"-":  r"\-",
                                                  "]":  r"\]",
                                                  "\\": r"\\",
                                                  "^":  r"\^",
                                                  "$":  r"\$",
                                                  "*":  r"\*",
                                                  ".":  r"\.",
                                                  "(":  r"\(",
                                                  ")":  r"\)" }))
    #print("ent_text=",ent_text)
    matches = re.finditer(ent_text, texto)
    local_ent = map(lambda match: (*match.span(), name), matches)
    entidades.extend(list(local_ent))
  entidades.sort()

  label_iden = {"total": False,
                "date": False,
                "address": False,
                "company": False
      }
  #print("entidades",entidades)
  entidades_n = []
  for ent in reversed(entidades):
    #print("ent=",ent)
    label= ent[2]
    if label_iden[label]==False:
      label_iden[label]=True
      entidades_n.append(ent)
    #print("entidades_n=",entidades_n)
  entidades_n.sort()
  #print("entidades_n",entidades_n)
  texto_faltantes=[]
  i=0
  pos_ini = 0
  pos_fin = 0
  #print("entidades_n=",entidades_n)
  for ent in entidades_n:
    if (i == 0) & (ent[0] > 0):
      pos_ini = 0
      pos_fin = ent[0]-1
      texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
      #print("ent=",ent,"pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
    if i < len(entidades_n)-1:
      if ent[1]+1 != entidades_n[i+1][0]:
        pos_ini = entidades_n[i][1]+1
        pos_fin = entidades_n[i+1][0]-1
        texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
        #print("ent=",ent,"pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
    i+=1

  if (pos_fin < len(texto)-1) & (entidades_n[-1][1] < len(texto)-1):
    if pos_fin > entidades_n[-1][1]:
      pos_ini = pos_fin + 1
      pos_fin = len(texto)
    else:
      pos_ini = entidades_n[-1][1]+1
      pos_fin = len(texto)
    texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
    #print("pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
  entidades_faltantes = []
  #print("texto_faltantes",texto_faltantes)
  for texto_fa in texto_faltantes:
    pos_ini,pos_fin,texto_f = texto_fa
    #print("pos_ini=",pos_ini," pos_fin=",pos_fin," texto_f=",texto_f)
    #tonkens = tokenizer(texto_f)
    tonkens = texto_f.split(' ')
    pos_ini_t = pos_ini
    for token in tonkens:
      pos_fin_t = pos_ini_t + len(token)
      #print("pos_ini_t=",pos_ini_t,"pos_fin_t=",pos_fin_t,"token=",token,"textoP=",texto[pos_ini_t:pos_fin_t])
      entidades_faltantes.append((pos_ini_t,pos_fin_t,'O'))
      pos_ini_t = pos_fin_t + 1
  #print("entidades_faltantes",entidades_faltantes)
  entidades_final=entidades_n + entidades_faltantes
  entidades_final.sort()
  spacy.training.offsets_to_biluo_tags(nlp.make_doc(texto), entidades_final)
  tags = offsets_to_biluo_tags(nlp.make_doc(texto), entidades_final)
  #print("tags=",tags)
  #doc_p = nlp.make_doc(texto)
  #for tok in doc_p:
  #  print("tok=",tok)
  entidades_final=dict(entities=entidades_final)
  entidades_final= (texto,entidades_final,entidades_n,archivo)
  print("entidades_final=",entidades_final)
  return(entidades_final)

##Validacion entidades

In [ ]:
def entidades_total_prueba_entidad (ruta,archivo):
  #ruta ='/tmp/SROIE2019v02/SROIE2019/train/'
  #archivo = 'X51007419197.txt'

  #print(archivo)
  with open(ruta + "box/" + archivo, encoding='utf-8', errors='ignore') as f:
    ocr = f.readlines()
  data = pd.DataFrame(
    list(map(parse, ocr)),
    columns=[*(f"coor{i}" for i in range(8)), "text"]
    )
  data=data.dropna()
  #print("data",data)
  texto = build_text(data)
  #print("texto",texto)

  with open(ruta + "entities/" + archivo, encoding='utf-8') as f:
    entities = json.load(f)
  #print("entities=",entities)
  #print("type_entities=",type(entities))
  entidades = []
  for name, ent_text in entities.items():
    if len(ent_text)==0:
      continue
    else:
      #ent_text=re.escape(ent_text)
      ent_text=ent_text.translate(str.maketrans({"-":  r"\-",
                                                  "]":  r"\]",
                                                  "\\": r"\\",
                                                  "^":  r"\^",
                                                  "$":  r"\$",
                                                  "*":  r"\*",
                                                  ".":  r"\.",
                                                  "(":  r"\(",
                                                  ")":  r"\)" }))
    #print("ent_text=",ent_text)
    matches = re.finditer(ent_text, texto)
    local_ent = map(lambda match: (*match.span(), name), matches)
    entidades.extend(list(local_ent))
  entidades.sort()
  #print("entidades",entidades)
  #print("type_entidades=",type(entidades))
  label_iden = {"total": False,
                "date": False,
                "address": False,
                "company": False
      }
  #print("entidades",entidades)
  entidades_n = []
  for ent in reversed(entidades):
    #print("ent=",ent)
    label= ent[2]
    if label_iden[label]==False:
      label_iden[label]=True
      entidades_n.append(ent)
  entidades_n.sort()
  #print("entidades_n=",entidades_n)
  entidadesnoen = []
  for name, ent_text in entities.items():
    #print(name)
    ent_e=0
    for elemeto in entidades_n:
      #print(elemeto[2])
      if name == elemeto[2]:
        ent_e+=1
    if ent_e == 0:
      entidadesnoen.append((name,archivo))
  if len(entidadesnoen)>0:
    print("entidadesnoen",entidadesnoen)
  return(entidadesnoen)

In [ ]:
def entidades_total_prueba (ruta,archivo):
  #ruta ='/tmp/SROIE2019v02/SROIE2019/train/'
  #archivo = 'X51007419197.txt'

  print(archivo)
  with open(ruta + "box/" + archivo, encoding='utf-8', errors='ignore') as f:
    ocr = f.readlines()
  data = pd.DataFrame(
    list(map(parse, ocr)),
    columns=[*(f"coor{i}" for i in range(8)), "text"]
    )
  data=data.dropna()
  #print("data",data)
  texto = build_text(data)
  #print("texto",texto)

  with open(ruta + "entities/" + archivo, encoding='utf-8') as f:
    entities = json.load(f)
  #print("entities=",entities)
  #print("type_entities=",type(entities))
  entidades = []
  for name, ent_text in entities.items():
    if len(ent_text)==0:
      continue
    else:
      ent_text=ent_text.translate(str.maketrans({"-":  r"\-",
                                                  "]":  r"\]",
                                                  "\\": r"\\",
                                                  "^":  r"\^",
                                                  "$":  r"\$",
                                                  "*":  r"\*",
                                                  ".":  r"\.",
                                                  "(":  r"\(",
                                                  ")":  r"\)" }))
    #print("ent_text=",ent_text)
    matches = re.finditer(ent_text, texto)
    local_ent = map(lambda match: (*match.span(), name), matches)
    entidades.extend(list(local_ent))
  entidades.sort()

  label_iden = {"total": False,
                "date": False,
                "address": False,
                "company": False
      }
  #print("entidades",entidades)
  entidades_n = []
  for ent in reversed(entidades):
    #print("ent=",ent)
    label= ent[2]
    if label_iden[label]==False:
      label_iden[label]=True
      entidades_n.append(ent)
    #print("entidades_n=",entidades_n)
  entidades_n.sort()
  #print("entidades_n",entidades_n)
  texto_faltantes=[]
  i=0
  pos_ini = 0
  pos_fin = 0

  for ent in entidades_n:
    if (i == 0) & (ent[0] > 0):
      pos_ini = 0
      pos_fin = ent[0]-1
      texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
      #print("if 1 ent=",ent,"pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
    if i < len(entidades_n)-1:
      if ent[1]+1 != entidades_n[i+1][0]:
        pos_ini = entidades_n[i][1]+1
        pos_fin = entidades_n[i+1][0]-1
        texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
        #print("if 2 ent=",ent,"pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
    i+=1

  if (pos_fin < len(texto)-1) & (entidades_n[-1][1] < len(texto)-1):
    if pos_fin > entidades_n[-1][1]:
      pos_ini = pos_fin + 1
      pos_fin = len(texto)
    else:
      pos_ini = entidades_n[-1][1]+1
      pos_fin = len(texto)
    texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
    #print("if 3 ent=",ent, "pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
  entidades_faltantes = []
  #print("texto_faltantes",texto_faltantes)
  for texto_fa in texto_faltantes:
    pos_ini,pos_fin,texto_f = texto_fa
    #print("pos_ini=",pos_ini," pos_fin=",pos_fin," texto_f=",texto_f)
    #tonkens = tokenizer(texto_f)
    tonkens = texto_f.split(' ')
    pos_ini_t = pos_ini
    for token in tonkens:
      pos_fin_t = pos_ini_t + len(token)
      #print("pos_ini_t=",pos_ini_t,"pos_fin_t=",pos_fin_t,"token=",token,"textoP=",texto[pos_ini_t:pos_fin_t])
      entidades_faltantes.append((pos_ini_t,pos_fin_t,'O'))
      pos_ini_t = pos_fin_t + 1
  #print("entidades_faltantes",entidades_faltantes)
  entidades_final=entidades_n + entidades_faltantes
  entidades_final.sort()
  spacy.training.offsets_to_biluo_tags(nlp.make_doc(texto), entidades_final)
  tags = offsets_to_biluo_tags(nlp.make_doc(texto), entidades_final)
  #print("tags=",tags)
  #doc_p = nlp.make_doc(texto)
  #for tok in doc_p:
   #print("tok=",tok)
  entidades_final=dict(entities=entidades_final)
  entidades_final= (texto,entidades_final,entidades_n)
  #print("entidades_final=",entidades_final)
  return(entidades_final)

Nueva funcion cargue

In [ ]:
def entidades_total_opc2 (ruta,archivo):
  print(archivo)
  with open(ruta + "box/" + archivo, encoding='utf-8', errors='ignore') as f:
    ocr = f.readlines()
  data = pd.DataFrame(
    list(map(parse, ocr)),
    columns=[*(f"coor{i}" for i in range(8)), "text"]
    )
  data=data.dropna()
  texto = build_text(data)
  with open(ruta + "entities/" + archivo, encoding='utf-8') as f:
    entities = json.load(f)
  entidades = []
  for name, ent_text in entities.items():
    if len(ent_text)==0:
      continue
    else:
      ent_text=re.escape(ent_text)
    matches = re.finditer(ent_text, texto)
    local_ent = map(lambda match: (*match.span(), name), matches)
    entidades.extend(list(local_ent))
  entidades.sort()
  label_iden = {"total": False,
                "date": False,
                "address": False,
                "company": False
      }
  print("entidades",entidades)
  entidades_n = []
  for ent in entidades:
    label= ent[2]
    if label_iden[label]==False:
      if (label=='date') & (label_iden['total']==True):
        continue
      else:
        label_iden[label]=True
        entidades_n.append(ent)
  print("entidades_n",entidades_n)
  texto_faltantes=[]
  entidades_faltantes = []
  i=0
  pos_ini = 0
  pos_fin = 0
  for ent in entidades_n:
    if (i == 0) & (ent[0] > 0):
      pos_ini = 0
      pos_fin = ent[0]-1
      texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
      entidades_faltantes.append((pos_ini,pos_fin,'O'))
    else:
      if i < len(entidades_n)-1:
        if ent[1]+1 != entidades_n[i+1][0]:
          pos_ini = entidades_n[i][1]+1
          pos_fin = entidades_n[i+1][0]-1
          texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
          entidades_faltantes.append((pos_ini,pos_fin,'O'))
    i+=1
  if (pos_fin < len(texto)-1) & (entidades_n[-1][1] < len(texto)-1):
    if pos_fin > entidades_n[-1][1]:
      pos_ini = pos_fin + 1
      pos_fin = len(texto)
    else:
      pos_ini = entidades_n[-1][1]+1
      pos_fin = len(texto)
    texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
    entidades_faltantes.append((pos_ini,pos_fin,'O'))

  print("texto_faltantes",texto_faltantes)
  print("entidades_faltantes",entidades_faltantes)
  entidades_final=entidades_n + entidades_faltantes
  entidades_final.sort()
  entidades_final=dict(entities=entidades_final)
  entidades_final= (texto,entidades_final,archivo)
  return(entidades_final)

##Armar Dataset

In [ ]:
data_dir = "/tmp/SROIE2019v02/SROIE2019/train/box"
data_dir = pathlib.Path(data_dir)
box_count = len(list(data_dir.glob('**/*.txt')))
print(box_count)
l_box=list(data_dir.glob('**/*.txt'))

data_dir = "/tmp/SROIE2019v02/SROIE2019/test/box"
data_dir = pathlib.Path(data_dir)
box_count_test = len(list(data_dir.glob('**/*.txt')))
print(box_count_test)
l_box_test=list(data_dir.glob('**/*.txt'))

626
347


In [ ]:
l_box_archivo= list(map(lambda cadena: str(cadena)[-16:], l_box))

In [ ]:
l_box_test_archivo= list(map(lambda cadena: str(cadena)[-16:], l_box_test))

Revisar entidades train

In [ ]:
resul= []
ruta ='/tmp/SROIE2019v02/SROIE2019/train/'
#archivo = 'X51006913060.txt'
#resul=entidades_total_prueba_entidad (ruta,archivo)
for archivo in l_box_archivo:
  resul=entidades_total_prueba_entidad (ruta,archivo)


Revisar entidades test

In [ ]:
resul= []
ruta ='/tmp/SROIE2019v02/SROIE2019/test/'
#archivo = 'X51006619842.txt'
#resul=entidades_total_prueba_entidad (ruta,archivo)
for archivo in l_box_test_archivo:
  resul=entidades_total_prueba_entidad (ruta,archivo)

Revisar spacy train

In [ ]:
resul= []
ruta ='/tmp/SROIE2019v02/SROIE2019/train/'
#archivo = 'X51005719904.txt'
#resul=entidades_total_prueba (ruta,archivo)
for archivo in l_box_archivo:
 resul=entidades_total_prueba (ruta,archivo)

X51007339656.txt
X51006311780.txt
X51005757349.txt
X51005676546.txt
X51006328920.txt
X51005757294.txt
X51006619550.txt
X51008145505.txt
X51005361946.txt
X51005442333.txt
X51006679222.txt
X51005719886.txt
X51005663272.txt
X51005806703.txt
X51005757273.txt
X51008142033.txt
X51006387847.txt
X51006401723.txt
X51005745188.txt
X51005677335.txt
X51005719902.txt
X51006388065.txt
X51006913074.txt
X51008164420.txt
X51007339657.txt
X51005757201.txt
X51007339157.txt
X51005724629.txt
X51005745296.txt
X51005453802.txt
X51005719917.txt
X51006414733.txt
X51005605285.txt
X51005442338.txt
X51008099087.txt
X51007846269.txt
X51006557184.txt
X51006311764.txt
X51005663278.txt
X51008114283.txt
X51005745214.txt
X51007231370.txt
X51007339097.txt
X51007846326.txt
X51006620176.txt
X51008142068.txt
X51006556839.txt
X51006327976.txt
X51006913059.txt
X51007339133.txt
X51008164999.txt
X51006335547.txt
X51005433514.txt
X51005447850.txt
X51006619758.txt
X51008123604.txt
X51006389893.txt
X51005719873.txt
X51005200938.t

Revisar spacy test

In [ ]:
resul= []
ruta ='/tmp/SROIE2019v02/SROIE2019/test/'
#archivo = 'X51006619842.txt'
#resul=entidades_total_prueba (ruta,archivo)
for archivo in l_box_test_archivo:
  resul=entidades_total_prueba (ruta,archivo)

X51005677327.txt
X51005568866.txt
X51006619869.txt
X51007231276.txt
X51006008081.txt
X51007846283.txt
X51007231275.txt
X51005719895.txt
X51008099100.txt
X51006619764.txt
X51006502534.txt
X51007339119.txt
X51006335818.txt
X51006828199.txt
X51006328937.txt
X51005806692.txt
X51006912976.txt
X51006557196.txt
X51006349083.txt
X51007846371.txt
X51005724552.txt
X51005568892.txt
X51006329388.txt
X51005724611.txt
X51006414532.txt
X51007339163.txt
X51006401853.txt
X51006387660.txt
X51007339121.txt
X51006912959.txt
X51005711444.txt
X51007339161.txt
X51005757292.txt
X51005746210.txt
X51007339159.txt
X51007846321.txt
X51005568887.txt
X51008042787.txt
X51005605332.txt
X51006554841.txt
X51005684949.txt
X51005433518.txt
X51005605287.txt
X51005337877.txt
X51006387950.txt
X51005677336.txt
X51006556806.txt
X51005724623.txt
X51005724626.txt
X51005433556.txt
X51007231331.txt
X51006349085.txt
X51005337867.txt
X51006619346.txt
X51005684911.txt
X51006619569.txt
X51005763999.txt
X51005746203.txt
X51007339125.t

In [ ]:
textos_nerd = joblib.Parallel(n_jobs=-1)(joblib.delayed(entidades_total_prueba)('/tmp/SROIE2019v02/SROIE2019/train/',archivo) for archivo in tqdm(l_box_archivo))

100%|██████████| 626/626 [05:18<00:00,  1.97it/s]


In [ ]:
textos_nerd_test = joblib.Parallel(n_jobs=-1)(joblib.delayed(entidades_total_prueba)('/tmp/SROIE2019v02/SROIE2019/test/',archivo) for archivo in tqdm(l_box_test_archivo))

100%|██████████| 347/347 [02:41<00:00,  2.15it/s]


##Parametros

In [ ]:
labels= ['company', 'date', 'address', 'total','O']

for label in labels:
    ner.add_label(label)

optimizer = nlp.begin_training()

epochs = 94
loss = 0
n_epoch = 500

In [ ]:
spans_train = textos_nerd
spans_test = textos_nerd_test

In [ ]:
print(spans_train)

[('AIK HUAT HARDWARE ENTERPRISE (SETIA ALAM) SDN BHD 822737-X NO. 17-G, JALAN SETIA INDAH (X) U13/X, SETIA ALAM, SEKSYEN U13, 40170 SHAH ALAM, TEL: 012 - 6651783 FAX: 03 - 33623608 GST NO: 000394528768 SIMPLIFIED TAX INVOICE CASH RECEIPT #: CSP0388298 DATE: 10/05/2017 SALESPERSON : AH002 TIME: 16:22:00 ITEM QTY U/P AMOUNT (RM) (RM) 120000143 1 16.00 16.00 S SCHNEIDER VIVACE 15A S/SOCKET TOTAL QUANTITY 1 SUB-TOTAL (GST) DISC ROUNDING TOTAL CASH CHANGE 16.00 0.00 0.00 16.00 16.00 0.00 *GST @ 6% INCLUDED IN TOTAL GST SUMMARY CODE AMOUNT % TAX/AMT SR 15.09 6 0.91 TAX TOTAL: 0.91 GOODS SOLD ARE NOT REFUNDABLE, THANK YOU FOR CHOOSING US. PLS PROVIDE ORIGINAL BILL FOR GOODS EXCHANGE WITHIN 1 WEEK FROM TRANSACTION GOODS MUST BE IN ORIGINAL STATE TO BE ENTITLED FOR EXCHANGE.', {'entities': [(0, 49, 'company'), (50, 58, 'O'), (59, 139, 'address'), (140, 144, 'O'), (145, 148, 'O'), (149, 150, 'O'), (151, 158, 'O'), (159, 163, 'O'), (164, 166, 'O'), (167, 168, 'O'), (169, 177, 'O'), (178, 181, 'O'

### Entrenamiento

In [ ]:
def metrics_ner(test, nlp):
    num = 0; matches = 0; Nomatches = 0
    metrics = []
    for sent, lab, arch in test:
        doc = nlp(sent)
        preds = list(map(lambda ent: ent.label_, doc.ents))
        y_true = list(map(lambda case: case[2], lab["entities"]))
        num += len(y_true)
        matches += sum(pred == true for pred, true in zip(preds, y_true))
        Nomatches += sum(pred != true for pred, true in zip(preds, y_true))
    TruePositive =  matches
    FalseNegatives = Nomatches

    accuracy = TruePositive / num
    precision = 0
    recall = 0
    f1Score = 0
    if TruePositive + FalseNegatives > 0:
      precision = TruePositive / (TruePositive + FalseNegatives)
      recall = TruePositive / (TruePositive + FalseNegatives)
      f1Score = 2 * precision * recall / (precision + recall)
    metrics.append(accuracy)
    metrics.append(precision)
    metrics.append(recall)
    metrics.append(f1Score)
    return metrics

def evaluate(ner_model, examples):
    scorer = Scorer()
    example = []
    for input_, annotations in examples:
        pred = ner_model(input_)
        #print('Predicion: ',pred,'Anotacion: ',annotations)
        temp = Example.from_dict(pred, dict.fromkeys(annotations))
        example.append(temp)
    scores = scorer.score(example)
    return scores

for epoch in range(epochs):
    losses = {}
    random.shuffle(spans_train)
    batches = minibatch(spans_train[:n_epoch], size=compounding(4, 32, 1.1))
    for batch in batches:
        examples = []
        for sentence in batch:
            example = Example.from_dict(
                nlp.make_doc(sentence[0]), sentence[1]
                )
            examples.append(example)
        nlp.update(
            examples,
            sgd=optimizer,
            drop=0.5,
            losses=losses,
            )
    mner = metrics_ner(spans_test, nlp)
    print(f"Epoch {epoch + 1:<5}/{epochs:<5}loss:{losses['ner'] - loss:.4f} acc:{mner[0]:.4f} pre:{mner[1]:.4f} recall:{mner[2]:.4f} F1:{mner[3]:.4f}")
    loss = losses["ner"]

Epoch 1    /200  loss:47210.1204 acc:0.9604 pre:0.9604 recall:0.9604 F1:0.9604
Epoch 2    /200  loss:-13701.3358 acc:0.8993 pre:0.9586 recall:0.9586 F1:0.9586
Epoch 3    /200  loss:-14832.0352 acc:0.9492 pre:0.9601 recall:0.9601 F1:0.9601
Epoch 4    /200  loss:-3528.8565 acc:0.9312 pre:0.9533 recall:0.9533 F1:0.9533
Epoch 5    /200  loss:-1911.4368 acc:0.9039 pre:0.9505 recall:0.9505 F1:0.9505
Epoch 6    /200  loss:-957.9418 acc:0.9113 pre:0.9485 recall:0.9485 F1:0.9485
Epoch 7    /200  loss:-174.7645 acc:0.9349 pre:0.9602 recall:0.9602 F1:0.9602
Epoch 8    /200  loss:-855.8256 acc:0.9248 pre:0.9590 recall:0.9590 F1:0.9590
Epoch 9    /200  loss:-520.6657 acc:0.9245 pre:0.9615 recall:0.9615 F1:0.9615
Epoch 10   /200  loss:-799.8528 acc:0.9280 pre:0.9620 recall:0.9620 F1:0.9620
Epoch 11   /200  loss:-4.6877 acc:0.9382 pre:0.9646 recall:0.9646 F1:0.9646
Epoch 12   /200  loss:-555.9558 acc:0.9310 pre:0.9585 recall:0.9585 F1:0.9585
Epoch 13   /200  loss:-505.8100 acc:0.9423 pre:0.9656 recal

###Valdiación NER

In [ ]:
texto='3-1708032 POPULAR BOOL CO. (M) SDN BHD (CO. NO. 113821-W) (GST REG NO. 001492992000) NO 8, JALAN 7/1188, BESA TAN RAZAK 56000 KUALA LUMPUR, MALAYSIA KSL CITY TEL: 017-7765076 / 7765987 06/02/18 18:47 SLIP NO.: 0010291725 CHEAH PEI NI TRANS: 246761 DESCRIPTION AMOUNT CORR. PEN ZLI-W 7.65 T PB F/ RING FILE W/CLI 6.99 T TOTAL RM INCL. OF GST 14.64 ROUNDING ADJ 0.01 TOTAL RM 14.65 CASH -50.00 CHANGE 35.35 ITEM COUNT GST SUMMARY AMOUNT (RM) TAX (RM) 2 T @ 6% 13.81 0.83 TOTAL SAVINGS 0.00 BE A POPULAR CARD MEMBER AND ENJOY SPECUAL DUSCOUNTS THANK YOU. PLEASE COME AGAUR. WWW.POPULAR.COM.MY BUY CHINESE BOOKS ONLINE WWW.POPULARONLINE.COM.MY'
doc = nlp(texto)
print(doc.text)
displacy.render(doc,style='ent',jupyter=True)

3-1708032 POPULAR BOOL CO. (M) SDN BHD (CO. NO. 113821-W) (GST REG NO. 001492992000) NO 8, JALAN 7/1188, BESA TAN RAZAK 56000 KUALA LUMPUR, MALAYSIA KSL CITY TEL: 017-7765076 / 7765987 06/02/18 18:47 SLIP NO.: 0010291725 CHEAH PEI NI TRANS: 246761 DESCRIPTION AMOUNT CORR. PEN ZLI-W 7.65 T PB F/ RING FILE W/CLI 6.99 T TOTAL RM INCL. OF GST 14.64 ROUNDING ADJ 0.01 TOTAL RM 14.65 CASH -50.00 CHANGE 35.35 ITEM COUNT GST SUMMARY AMOUNT (RM) TAX (RM) 2 T @ 6% 13.81 0.83 TOTAL SAVINGS 0.00 BE A POPULAR CARD MEMBER AND ENJOY SPECUAL DUSCOUNTS THANK YOU. PLEASE COME AGAUR. WWW.POPULAR.COM.MY BUY CHINESE BOOKS ONLINE WWW.POPULARONLINE.COM.MY


In [ ]:
texto='RESTORAN WAN SHENG 002043319-W NO.2, JALAN TEMENGGUNG 19/9, SEKSYEN 9, BANDAR MAHKOTA CHERAS, 43200 CHERAS, SELANGOR GST REG NO: 001335787520 TAX INVOICE INV NO.: 1136284 CASHIER: NICOLE DATE : 11-05-2018 17:12:34 DESCRIPTION QTY U.PRICE TOTAL TAX BUNGA KEKWA 1 X 1.70 1.70 SR TAKE AWAY 1 X 0.20 0.20 SR TOTAL QTY: 2 TOTAL (EXCLUDING GST): 1.79 GST PAYABLE (6%): 0.11 TOTAL (INCLUSIVE OF GST): 1.90 TOTAL : 1.90 CASH : 1.90 GST SUMMARY AMOUNT(RM) TAX(RM) SR (@ 6%) 1.79 0.11'
doc = nlp(texto)
print(doc.text)
displacy.render(doc,style='ent',jupyter=True)

RESTORAN WAN SHENG 002043319-W NO.2, JALAN TEMENGGUNG 19/9, SEKSYEN 9, BANDAR MAHKOTA CHERAS, 43200 CHERAS, SELANGOR GST REG NO: 001335787520 TAX INVOICE INV NO.: 1136284 CASHIER: NICOLE DATE : 11-05-2018 17:12:34 DESCRIPTION QTY U.PRICE TOTAL TAX BUNGA KEKWA 1 X 1.70 1.70 SR TAKE AWAY 1 X 0.20 0.20 SR TOTAL QTY: 2 TOTAL (EXCLUDING GST): 1.79 GST PAYABLE (6%): 0.11 TOTAL (INCLUSIVE OF GST): 1.90 TOTAL : 1.90 CASH : 1.90 GST SUMMARY AMOUNT(RM) TAX(RM) SR (@ 6%) 1.79 0.11


In [ ]:
for ent in doc.ents:
  if ent.label_ != 'O':
    print('Entidad: ',ent.label_,' Valor: ',ent.text)

Entidad:  company  Valor:  RESTORAN WAN SHENG
Entidad:  address  Valor:  NO.2, JALAN TEMENGGUNG 19/9, SEKSYEN 9, BANDAR MAHKOTA CHERAS, 43200 CHERAS, SELANGOR
Entidad:  date  Valor:  11-05-2018
Entidad:  total  Valor:  1.90


###Guardar el Modelo

In [ ]:
joblib.dump(nlp, '/content/drive/MyDrive/modelo_ner_jdmc.pkl') # Guardo el modelo.
pnlp = joblib.load('/content/drive/MyDrive/modelo_ner_jdmc.pkl') # Carga del modelo.

In [ ]:
texto='RESTORAN WAN SHENG 002043319-W NO.2, JALAN TEMENGGUNG 19/9, SEKSYEN 9, BANDAR MAHKOTA CHERAS, 43200 CHERAS, SELANGOR GST REG NO: 001335787520 TAX INVOICE INV NO.: 1136284 CASHIER: NICOLE DATE : 11-05-2018 17:12:34 DESCRIPTION QTY U.PRICE TOTAL TAX BUNGA KEKWA 1 X 1.70 1.70 SR TAKE AWAY 1 X 0.20 0.20 SR TOTAL QTY: 2 TOTAL (EXCLUDING GST): 1.79 GST PAYABLE (6%): 0.11 TOTAL (INCLUSIVE OF GST): 1.90 TOTAL : 1.90 CASH : 1.90 GST SUMMARY AMOUNT(RM) TAX(RM) SR (@ 6%) 1.79 0.11'
doc = nlp(texto)
print(doc.text)
displacy.render(doc,style='ent',jupyter=True)

RESTORAN WAN SHENG 002043319-W NO.2, JALAN TEMENGGUNG 19/9, SEKSYEN 9, BANDAR MAHKOTA CHERAS, 43200 CHERAS, SELANGOR GST REG NO: 001335787520 TAX INVOICE INV NO.: 1136284 CASHIER: NICOLE DATE : 11-05-2018 17:12:34 DESCRIPTION QTY U.PRICE TOTAL TAX BUNGA KEKWA 1 X 1.70 1.70 SR TAKE AWAY 1 X 0.20 0.20 SR TOTAL QTY: 2 TOTAL (EXCLUDING GST): 1.79 GST PAYABLE (6%): 0.11 TOTAL (INCLUSIVE OF GST): 1.90 TOTAL : 1.90 CASH : 1.90 GST SUMMARY AMOUNT(RM) TAX(RM) SR (@ 6%) 1.79 0.11
